http://fho.f12n.de/posts/2014-05-07-dont-fear-the-cat.html

In [1]:
{-# LANGUAGE DeriveFunctor #-}
{-# LANGUAGE TypeFamilies #-}

In [2]:
import Data.Functor.Foldable 
import Data.List (splitAt, unfoldr)

In [3]:
data TreeF c f = EmptyF | LeafF c | NodeF f f
   deriving (Eq, Show, Functor)

In [4]:
unflatten :: [a] -> TreeF a [a]
unflatten (  []) = EmptyF
unflatten (x:[]) = LeafF x
unflatten (  xs) = NodeF l r   
   where (l,r) = splitAt (length xs `div` 2) xs

In [5]:
example1 = ana unflatten [1,3] 
:t example1
--Fix (NodeF (Fix (LeafF 1)) (Fix (LeafF 3)))

example1 :: forall a t. (Base t ~ TreeF a, Corecursive t, Num a) => t

In [6]:
flatten :: Ord a => TreeF a [a] -> [a]
flatten EmptyF      = []
flatten (LeafF c)   = [c]
flatten (NodeF l r) = mergeSortedLists l r 

mergeSortedLists :: Ord a => [a] -> [a] -> [a]
mergeSortedLists = curry $ unfoldr c where
   c ([], [])     = Nothing
   c ([], y:ys)   = Just (y, ([], ys))
   c (x:xs, [])   = Just (x, (xs, []))
   c (x:xs, y:ys) | x <= y = Just (x, (xs, y:ys)) 
                  | x > y  = Just (y, (x:xs, ys))

In [7]:
example2 = 
  cata flatten (Fix (NodeF (Fix (LeafF 3)) (Fix (LeafF 1)))) -- == [1,3]
print example2

[1,3]

In [8]:
mergeSort :: Ord a => [a] -> [a]
mergeSort = hylo flatten unflatten

example3 = mergeSort [5,2,7,9,1,4] -- == [1,2,4,5,7,9] 
print example3
mergeSort [2, 5, 8, 4, 7, 3, 1, 77, 3, 2, 55, 0]


[1,2,4,5,7,9]

[0,1,2,2,3,3,4,5,7,8,55,77]

In [9]:
data Tree c = Empty | Leaf c | Node (Tree c) (Tree c)
   deriving (Eq, Show)

In [10]:
import Data.Functor.Foldable

type instance Base (Tree c) = (TreeF c)
instance Corecursive (Tree c) where --Unfoldable
   embed EmptyF      = Empty
   embed (LeafF c)   = Leaf c
   embed (NodeF l r) = Node l r
instance Recursive (Tree c) where -- Foldable
   project Empty      = EmptyF
   project (Leaf c)   = LeafF c
   project (Node l r) = NodeF l r


In [11]:
example4 = cata flatten (Node (Leaf 'l') (Leaf 'r')) -- == "lr"
print example4

"lr"

In [12]:
unflat :: [a] -> Tree a 
unflat (  []) = Empty
unflat (x:[]) = Leaf x
unflat (  xs) = Node (unflat l) (unflat r) -- originally: NodeF l r   
   where (l,r) = splitAt (length xs `div` 2) xs
   
--unflat (Node (Node (Leaf 3)(Leaf 7)) (Node Empty (Leaf 4)))
unflat [3,7,4]

Node (Leaf 3) (Node (Leaf 7) (Leaf 4))

In [14]:
flatt :: Ord a => Tree a -> [a]
flatt Empty      = []
flatt (Leaf c)   = [c]
flatt (Node l r) = mergeSortedLists (flatt l) (flatt r ) -- originally: mergeSortedLists l r 

flatt (Node (Leaf 3) (Node (Leaf 7) (Leaf 4)))

[3,4,7]

In [18]:
tr = flatt . unflat 
:t tr
tr [3,7,4]

tr :: forall a. Ord a => [a] -> [a]

[3,4,7]

In [22]:
tr2 = flatten . unflatten
tr2 [3,4,7]

[3,4,7]

In [28]:
--hyloBTree h g = cataBTree h . anaBTree g

:t ana
:t cata
:t hylo -- hylo~f~h~=~cata~f~\circ~ana~h


ana :: forall t a. Corecursive t => (a -> Base t a) -> a -> t

cata :: forall a t. Recursive t => (Base t a -> a) -> t -> a

hylo :: forall a (f :: * -> *) b. Functor f => (f b -> b) -> (a -> f a) -> a -> b